In [1]:
## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms

from torch import nn
from torch import optim
from jetnet_dataloader import JetNetDataloader
import math
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import traceback
import os
import pytorch_lightning as pl
from torch.optim.lr_scheduler import OneCycleLR,ReduceLROnPlateau,ExponentialLR
import torch
from torch import nn
from torch.nn import functional as FF
import numpy as np
from jetnet.evaluation import w1p, w1efp, w1m, cov_mmd,fpnd
import mplhep as hep
import hist
from hist import Hist
from pytorch_lightning.loggers import TensorBoardLogger
from collections import OrderedDict
from ray import tune
from helpers import *
from plotting import *
import pandas as pd
import os
from torch.utils.data import DataLoader
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import time
import pytorch_lightning as pl

from torch import nn
from torch.nn import functional as FF
import numpy as np
from jetnet.evaluation import w1p, w1efp, w1m, cov_mmd,fpnd
import mplhep as hep
import hist
from hist import Hist
from pytorch_lightning.loggers import TensorBoardLogger
from torch.autograd import Variable
from helpers import *
from plotting import *
import pandas as pd
import os
from pytorch_lightning.loggers import TensorBoardLogger
import datetime
from torch.autograd import Variable

thx max
good boy


In [17]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from nflows.distributions.base import Distribution
from sklearn.model_selection import train_test_split
from torch import optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import PowerTransformer
from helpers import *


class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)
    def inverse_transform(self,values):
        return (values *self.std)+self.mean
    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)
    def to(self,dev):
        self.std=self.std.to(dev)
        self.mean=self.mean.to(dev)
        return self
  
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


class JetNetDataloader(pl.LightningDataModule):
    '''This is more or less standard boilerplate coded that builds the data loader of the training
       one thing to note is the custom standard scaler that works on tensors
       Currently only jets with 30 particles are used but this maybe changes soon'''
    def __init__(self,config): 
        super().__init__()
        self.config=config
        self.n_dim=config["n_dim"]
        self.n_part=config["n_part"]
        self.batch_size=config["batch_size"]
    def setup(self,stage):
    # This just sets up the dataloader, nothing particularly important. it reads in a csv, calculates mass and reads out the number particles per jet
    # And adds it to the dataset as variable. The only important thing is that we add noise to zero padded jets
        data_dir=os.environ["HOME"]+"/JetNet_NF/train_{}_jets.csv".format(self.config["parton"])
        data=pd.read_csv(data_dir,sep=" ",header=None)
        jets=[]
        df=pd.DataFrame()
        limit=int(self.config["limit"]*1.1)

            #masks=np.sum(data.values[:,np.arange(3,120,4)],axis=1)
        masks=data.values[:,np.arange(3,120,4)][:self.config["limit"]]
        df=data.drop(np.arange(3,120,4),axis=1)[:self.config["limit"]]

        #stacking together differnet samples with different number particles per jet
    
      
        # calculating mass per jet
#         self.m=mass(self.data[:,:self.n_dim]).reshape(-1,1)  
      # Adding noise to zero padded jets.
        z=torch.tensor(df.values).reshape(len(df),30,3)
        m=(torch.tensor(masks).reshape(len(df),30)).bool()


        self.data=z
        self.data[~m,:]=torch.normal(mean=torch.zeros_like(self.data[~m,:]),std=1).abs()*1e-7
        print(self.data.reshape(-1,30,3)[:,:,2].min())
        #standard scaling 
        self.scaler=StandardScaler()

#         self.data=torch.hstack((self.data,self.m))        
        self.scaler.fit(self.data[:,:,:2])
        self.ptscaler=PowerTransformer("box-cox")
        self.data[:,:,:2]=self.scaler.transform(self.data[:,:,:2])
        
        self.data[:,:,2]=torch.tensor(self.ptscaler.transform(self.data[:,:,2].numpy()))
        print(self.ptscaler.inverse_transform(self.data[:,:,2]).min())
        print(self.ptscaler.inverse_transform(np.zeros((10,30))+self.data[:10,:,2].numpy().min()*0.9))
        #         self.min_m=self.scaler.transform(torch.zeros((1,self.n_dim+1)))[0,-1]
# #         self.data=torch.hstack((self.data,self.n))
        
#         #calculating mass dist in different bins, this is needed for the testcase where we need to generate the conditoon
#         if self.config["variable"]:
#             self.mdists={}
#             for i in torch.unique(self.n):
#                 self.mdists[int(i)]=F(self.data[self.n[:,0]==i,-2])    
        self.data,self.test_set=train_test_split(self.data.cpu().numpy(),test_size=0.3)
        
#         self.n_train=self.data[:,-1]
#         self.n_test=self.test_set[:,-1]
        
            
        self.test_set=torch.tensor(self.test_set).float()
        self.data=torch.tensor(self.data).float()
        self.num_batches=len(self.data)//self.config["batch_size"]
#         assert self.data.shape[1]==92
        assert (torch.isnan(self.data)).sum()==0
    def train_dataloader(self):
        return DataLoader(self.data, batch_size=self.batch_size,drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=len(self.test_set),drop_last=True)
config={"n_dim":3,"n_part":30,"parton":"t","limit":150000,"batch_size":512}
JetNetDataloader(config).setup(stage="train")

tensor(6.1953e-11, dtype=torch.float64)


ValueError: The Box-Cox transformation can only be applied to strictly positive data

In [3]:
import matplotlib.pyplot as plt
import os
import hist
import mplhep as hep
import torch
import numpy as np
import hist
from hist import Hist
import traceback
from helpers import mass
import pandas as pd
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor
from mpl_toolkits.axes_grid1 import make_axes_locatable
class plotting():
    '''This is a class that takes care of  plotting steps in the script,
        It is initialized with the following arguments:
        true=the simulated data, note that it needs to be scaled
        gen= Generated data , needs to be scaled
        step=The current step of the training, this is need for tensorboard
        model=the model that is trained, a bit of an overkill as it is only used to access the losses
        config=the config used for training
        logger=The logger used for tensorboard logging'''
    def __init__(self,true,gen,step,model=None,logger=None,weight=1):

        self.n_dim=90
        self.gen=gen
        self.test_set=true
        self.step=step
        self.model=model

        self.weight=weight
        if logger is not None:
            self.summary=logger
    def plot_mass_only(self,m,m_t,bins=15):
        fig,ax=plt.subplots(2,1,gridspec_kw={'height_ratios': [3, 1]},figsize=(6,8))
        a=min(np.quantile(m_t,0.001),np.quantile(m,0.001))
        b=max(np.quantile(m_t,0.999),np.quantile(m,0.999))
        a=np.quantile(m_t,0.001)
        b=np.quantile(m_t,0.999)
        h=hist.Hist(hist.axis.Regular(bins,a,b))
        h2=hist.Hist(hist.axis.Regular(bins,a,b))
        bins = h.axes[0].edges
        h.fill(m)#,weight=1/self.weight)
        h2.fill(m_t)
            
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )

        main_ax_artists, sublot_ax_arists = h.plot_ratio(
            h2,
            ax_dict={"main_ax":ax[0],"ratio_ax":ax[1]},
            rp_ylabel=r"Ratio",
            rp_num_label="Flow Generated",
            rp_denom_label="MC Simulatied",
            rp_uncert_draw_type="line",  # line or bar
        )
        ax[0].set_xlabel("")
#                 if quantile and v=="m" and plot_vline:
#                     ax[0,k].hist(m[m_t<np.quantile(m_t,0.1)],histtype='step',bins=bins,alpha=1,color="red",label="10% quantile gen",hatch="/")
#                     ax[0,k].vlines(np.quantile(m_t,0.1),0,np.max(h[:]),color="red",label='10% quantile train')

        ax[1].set_ylim(0.25,2)
        ax[0].set_xlim(a,b)
        ax[1].set_xlabel("$m_T$",fontweight="bold")
        ax[1].set_xlim(a,b)
        ax[0].set_ylabel("Counts",fontweight="bold" )
        ax[1].set_ylabel("Ratio",fontweight="bold")
  
     
#             print("added figure")
#             self.summary.close()

        plt.savefig("{}{}{}_mass".format(self.typ,self.c,self.p))
        plt.show()

    def plot_marginals(self,ith=None,save=True,title=None):
        #This plots the marginal distribution for simulation and generation
        #Note that this is the data the model sees during training as input to model in the NF
        #This is the distribution of one of [eta,phi,pt] of one particle of the n particles per jet: for example the pt of the 3rd particle
        #if save, the histograms are logged to tensorboard otherwise they are shown
        
        plt.switch_backend('agg')
        name,label=["eta","phi","pt"],['${\eta}^{rel}_{7}$',"${\phi}^{rel}_{7}$","${p_T}^{rel}_{7}$"]
        fig,ax=plt.subplots(2,3,gridspec_kw={'height_ratios': [3, 1]},figsize=(18,6))
        particles=range(self.n_dim) if not ith else [3*ith,3*ith+1,3*ith+2]
        plt.suptitle(title,fontweight="bold")
        k=0
        for i in particles:
            if ith:
                ax_temp=ax[:,k]
            else:
                fig,ax_temp=plt.subplots(2,1)
            a=np.quantile(self.test_set[:,i].numpy(),0)
            b=np.quantile(self.test_set[:,i].numpy(),1)

            h=hist.Hist(hist.axis.Regular(15,a,b,label=label[i%3],underflow=False,overflow=False))
            h2=hist.Hist(hist.axis.Regular(15,a,b,label=label[i%3],underflow=False,overflow=False))
            h.fill(self.gen[:,i].numpy())
            h2.fill(self.test_set[:,i].numpy())
            
            plt.tight_layout()
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0,k] )
       
            main_ax_artists, sublot_ax_arists = h.plot_ratio(
                h2,
                ax_dict={"main_ax":ax_temp[0],"ratio_ax":ax_temp[1]},
                rp_ylabel=r"Ratio",
#                 rp_xlabel=label[i%3],
                rp_num_label="Flow Generated",
                rp_denom_label="MC Simulated",
                rp_uncert_draw_type="line",  # line or bar
            )
            
            
            ax_temp[0].set_xlabel("")
            ax_temp[1].set_ylim(0.25,2)
            ax_temp[0].set_xlim(a,b)
            ax_temp[1].set_xlim(a,b)
            ax_temp[1].set_xlabel(label[i%3])
            ax_temp[0].set_ylabel("Counts",fontweight="bold" )
            ax_temp[1].set_ylabel("Ratio",fontweight="bold")
            
            
            #plt.tight_layout(pad=2)
            k+=1
        if save:
            self.summary.add_figure("jet{}_{}".format(i//3+1,name[i%3]),fig,global_step=self.step)
            self.summary.close()
        else:
            plt.savefig("{}{}{}_7thpart".format(self.typ,self.c,self.p))
            plt.show()


    def plot_2d(self,save=True):
        #This creates a 2D histogram of the inclusive distribution for all 3 feature combinations
        #Inclusive means that is the distribution of pt of all particles per jet and sample
        #if save, the histograms are logged to tensorboard otherwise they are shown
        data=self.test_set[:,:self.n_dim].reshape(-1,3).numpy()
        gen=self.gen[:,:self.n_dim].reshape(-1,3).numpy()
        labels=[r"$\eta^{rel}$",r"$\phi^{rel}_7$",r"$p_T^{rel}$"]
        names=["eta","phi","pt"]
        for index in [[0,1],[0,2],[1,2]]:
            
            fig,ax=plt.subplots(ncols=2,figsize=(16, 8))
            _,x,y,_=ax[0].hist2d(data[:,index[0]],data[:,index[1]],bins=30)
            #rebin to only take 5% to 95.0% of signal dis
            a=np.quantile(x,0.05)
            b=np.quantile(x,0.95)
            x=np.linspace(a,b,len(x))
            a=np.quantile(y,0.05)
            b=np.quantile(y,0.95)
            y=np.linspace(a,b,len(y))
            if index[1]==2:
                y=np.abs(y)+0.00001
                y = np.logspace(np.log(y[0]),np.log(y[-1]),len(y))
            ax[0].hist2d(data[:,index[0]],data[:,index[1]],bins=[x,y])
            data[:,index[0]]=np.abs(data[:,index[0]])+0.00001
            ax[1].hist2d(gen[:,index[0]],gen[:,index[1]],bins=[x,y])
        
        
            plt.tight_layout(pad=2)
            ax[0].set_xlabel( labels[index[0]],fontweight="bold")
            ax[0].set_ylabel( labels[index[1]],fontweight="bold")
            
            ax[0].set_title("MC Simulated")
            ax[1].set_xlabel( labels[index[0]],fontweight="bold")
            ax[1].set_ylabel( labels[index[1]],fontweight="bold")
            
            ax[1].set_title("Flow Generated")
           
            if save:
                self.summary.add_figure("2d{}-{}".format(names[index[0]],names[index[1]]),fig,global_step=self.step)
                
                # self.summary.close()
            else:
                plt.savefig("{}{}{}_2dcorr{}{}".format(self.typ,self.c,self.p,names[index[0]],names[index[0]]))
                plt.show()
                
 
        
    def oversample(self,m,m_t,weight,save=True,quantile=False,bins=15,plot_vline=False,title=""):
        #This creates a histogram of the inclusive distributions and calculates the mass of each jet
        #and creates a histogram of that
        #if save, the histograms are logged to tensorboard otherwise they are shown
        #if quantile, this also creates a histogram of a subsample of the generated data, 
        # where the mass used to condition the flow is in the first 10% percentile of the simulated mass dist
        i=0
        k=0
        fig,ax=plt.subplots(2,4,gridspec_kw={'height_ratios': [3, 1]},figsize=(20,5))
        plt.suptitle(title,fontweight="bold")
        for v,name in zip(["eta","phi","pt","m"],[r"$\eta^{rel}$",r"$\phi^{rel}$",r"$p_T^{rel}$",r"$m^{rel}$"]):
            
            if v!="m":
                a=min(np.quantile(self.gen[:,i],0.001),np.quantile(self.test_set[:,i],0.001))
                b=max(np.quantile(self.gen[:,i],0.999),np.quantile(self.test_set[:,i],0.999))     
                
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                h.fill(self.gen[:,i],weight=1/weight)
                h2.fill(self.test_set[:,i])
                i+=1
            else:
                a=min(np.quantile(m_t,0.001),np.quantile(m,0.001))
                b=max(np.quantile(m_t,0.999),np.quantile(m,0.999))
                a=np.quantile(m_t,0.001)
                b=np.quantile(m_t,0.999)
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                bins = h.axes[0].edges
                h.fill(m,weight=1/weight)#,weight=1/self.weight)
                h2.fill(m_t)
            
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            try:
                main_ax_artists, sublot_ax_arists = h.plot_ratio(
                    h2,
                    ax_dict={"main_ax":ax[0,k],"ratio_ax":ax[1,k]},
                    rp_ylabel=r"Ratio",
                    rp_num_label="Flow Generated",
                    rp_denom_label="MC Simulatied",
                    rp_uncert_draw_type="line",  # line or bar
                )
                ax[0,k].set_xlabel("")
#                 if quantile and v=="m" and plot_vline:
#                     ax[0,k].hist(m[m_t<np.quantile(m_t,0.1)],histtype='step',bins=bins,alpha=1,color="red",label="10% quantile gen",hatch="/")
#                     ax[0,k].vlines(np.quantile(m_t,0.1),0,np.max(h[:]),color="red",label='10% quantile train')
                    
                ax[1,k].set_ylim(0.25,2)
                ax[0,k].set_xlim(a,b)
                ax[1,k].set_xlabel(name,fontweight="bold")
                ax[1,k].set_xlim(a,b)
                ax[0,k].set_ylabel("Counts",fontweight="bold" )
                ax[1,k].set_ylabel("Ratio",fontweight="bold")
                
#                 if plot_vline:
#                        ax[0,k].legend(["Generated","Training","10% quantile Gen","10% quantile Sim"] )
#                 else:
#                       ax[0,k].legend(["Flow Generated","MC Simulated"] )
            except:
                print("mass plot failed reverting to simple plot mass bins")
                plt.close()
                plt.figure()
                _,b,_=plt.hist(m_t,15,label="MC Simulated",alpha=0.5)
                plt.hist(m,b,label="Flow Generated",alpha=0.5)
                plt.legend()  
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            
#             plt.xlabel(name)
            plt.tight_layout(pad=1)
            k+=1
        if save:
            if v!="m":
                 self.summary.add_figure("inclusive"+v,fig,self.step)
            else:
                self.summary.add_figure("jet_mass",fig,self.step)
#             print("added figure")
#             self.summary.close()
        else:
            plt.savefig("{}{}{}_oversample_{}".format(self.typ,self.c,self.p,v))
            plt.show()

            
    def plot_mass(self,m,m_t,save=True,quantile=False,bins=15,plot_vline=False,title=""):
        #This creates a histogram of the inclusive distributions and calculates the mass of each jet
        #and creates a histogram of that
        #if save, the histograms are logged to tensorboard otherwise they are shown
        #if quantile, this also creates a histogram of a subsample of the generated data, 
        # where the mass used to condition the flow is in the first 10% percentile of the simulated mass dist
        i=0
        k=0
        fig,ax=plt.subplots(2,4,gridspec_kw={'height_ratios': [3, 1]},figsize=(20,5))
        plt.suptitle(title,fontweight="bold")
        for v,name in zip(["eta","phi","pt","m"],[r"$\eta^{rel}$",r"$\phi^{rel}$",r"$p_T^{rel}$",r"$m^{rel}$"]):
            
            if v!="m":
                a=min(np.quantile(self.gen[:,i],0.001),np.quantile(self.test_set[:,i],0.001))
                b=max(np.quantile(self.gen[:,i],0.999),np.quantile(self.test_set[:,i],0.999))     
                
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                h.fill(self.gen[:,i],weight=1/self.weight)
                h2.fill(self.test_set[:,i])
                i+=1
            else:
                a=min(np.quantile(m_t,0.001),np.quantile(m,0.001))
                b=max(np.quantile(m_t,0.999),np.quantile(m,0.999))
                a=np.quantile(m_t,0.001)
                b=np.quantile(m_t,0.999)
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                bins = h.axes[0].edges
                h.fill(m)#,weight=1/self.weight)
                h2.fill(m_t)
            
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            try:
                main_ax_artists, sublot_ax_arists = h.plot_ratio(
                    h2,
                    ax_dict={"main_ax":ax[0,k],"ratio_ax":ax[1,k]},
                    rp_ylabel=r"Ratio",
                    rp_num_label="Flow Generated",
                    rp_denom_label="MC Simulatied",
                    rp_uncert_draw_type="line",  # line or bar
                )
                ax[0,k].set_xlabel("")
#                 if quantile and v=="m" and plot_vline:
#                     ax[0,k].hist(m[m_t<np.quantile(m_t,0.1)],histtype='step',bins=bins,alpha=1,color="red",label="10% quantile gen",hatch="/")
#                     ax[0,k].vlines(np.quantile(m_t,0.1),0,np.max(h[:]),color="red",label='10% quantile train')
                    
                ax[1,k].set_ylim(0.25,2)
                ax[0,k].set_xlim(a,b)
                ax[1,k].set_xlabel(name,fontweight="bold")
                ax[1,k].set_xlim(a,b)
                ax[0,k].set_ylabel("Counts",fontweight="bold" )
                ax[1,k].set_ylabel("Ratio",fontweight="bold")
                
#                 if plot_vline:
#                        ax[0,k].legend(["Generated","Training","10% quantile Gen","10% quantile Sim"] )
#                 else:
#                       ax[0,k].legend(["Flow Generated","MC Simulated"] )
            except:
                print("mass plot failed reverting to simple plot mass bins")
                plt.close()
                plt.figure()
                _,b,_=plt.hist(m_t,15,label="MC Simulated",alpha=0.5)
                plt.hist(m,b,label="Flow Generated",alpha=0.5)
                plt.legend()  
            #hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            
#             plt.xlabel(name)
            plt.tight_layout(pad=1)
            k+=1
        if save:
            if v!="m":
                 self.summary.add_figure("inclusive"+v,fig,self.step)
            else:
                self.summary.add_figure("jet_mass",fig,self.step)
#             print("added figure")
#             self.summary.close()
        else:
            plt.savefig("inclusive_{}".format(v))
            plt.show()


#     def losses(self,save=False):
#         '''This plots the different losses vs epochs'''
#         fig=plt.figure()
#         hep.cms.label("Private Work",data=None,lumi=None,year=None)
#         plt.xlabel('step')
#         plt.ylabel('loss')
#         ln1=plt.plot(self.model.logprobs,label='log$(p_{gauss}(x_{data}))$')
#         if "calc_massloss" in self.config.keys() and self.config["calc_massloss"]:
#             plt.twinx()
#             ln2=plt.plot(self.model.mlosses,label=r'mass mse $\times$ {}'.format(self.config["lambda"]),color='orange')
#             plt.ylabel("MSE")
#             plt.yscale("log")
#             ln1+=ln2
#         labs=[l.get_label() for l in ln1]
#         plt.legend(ln1,labs)
#         plt.tight_layout(pad=2)
#         if save:
#             self.summary.add_figure("losses",fig,self.step)
# #             self.summary.close()
#         else:
#             plt.show()
   

    def plot_correlations(self,save=True):
        #Plots correlations between all particles for i=0 eta,i=1 phi,i=2 pt
        self.plot_corr(i=0,save=save)
        self.plot_corr(i=1,save=save)
        self.plot_corr(i=2,save=save)

    def plot_corr(self,i=0,names=["$\eta^{rel}$","$\phi^{rel}$","$p_T$"],save=True):
        if i==2:
            c=1
        else:
            c=0.3
        df_g=pd.DataFrame(self.gen[:,:self.n_dim].detach().numpy()[:,range(i,90,3)])
        df_h=pd.DataFrame(self.test_set[:,:self.n_dim].detach().numpy()[:,range(i,90,3)])
        fig,ax=plt.subplots(ncols=2,figsize=(20,10))
        corr_g = ax[0].matshow(df_g.corr())
        corr_g.set_clim(-c,c)
        divider = make_axes_locatable(ax[0])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar=fig.colorbar(corr_g,cax=cax)
        corr_h = ax[1].matshow(df_h.corr())
        corr_h.set_clim(-c,c)
        divider = make_axes_locatable(ax[1])
        cax2 = divider.append_axes('right', size='5%', pad=0.05)
        cbar=fig.colorbar(corr_h,cax=cax2)
        plt.suptitle("{} Correlation between Particles".format(names[i]),fontweight="bold")
        ax[0].set_title("Flow Generated",fontweight="bold")
        ax[1].set_title("MC Simulated",fontweight="bold")
        ax[0].set_xlabel("Particles",fontweight="bold")
        ax[0].set_ylabel("Particles",fontweight="bold")
        ax[1].set_xlabel("Particles",fontweight="bold")
        ax[1].set_ylabel("Particles",fontweight="bold")
        title=["corr_eta","corr_phi","corr_pt"]
        if save:
                
                self.summary.add_figure(title[i],fig,self.step)    
    #             self.summary.close()
        else:
                #plt.savefig("corr_{}".format(title[i]))
                plt.show()

    def var_part(self,true,gen,true_n,gen_n,m_true,m_gen,form=2,save=True):
        labels=["$\eta^{rel}$","$\phi^{rel}$","$p^{rel}_T$","$m^{rel}$"]
        names=["eta","phi","pt","m"]
        n,counts=torch.unique(true_n,return_counts=True)
        for j in range(4):
            fig,ax=plt.subplots(ncols=2,nrows=2,figsize=(15,15))

            k=-1
            ntemp=n[-form**2:]

            
            for i in list(ntemp)[::-1]: 
                k+=1
                i=int(i)

                if names[j]!="m":
                    a=np.quantile(self.test_set[true_n.reshape(-1)==i,:].reshape(-1,3)[:,j],0.001)
                    b=np.quantile(self.test_set[true_n.reshape(-1)==i,:].reshape(-1,3)[:,j],0.999)    
                    h=hist.Hist(hist.axis.Regular(15,a,b))
                    h2=hist.Hist(hist.axis.Regular(15,a,b))
                    bins = h.axes[0].edges

                    ax[k//form,k%form].legend()
                    h.fill(self.gen[gen_n.reshape(-1)==i,:].reshape(-1,3)[:,j])
                    h2.fill(self.test_set[true_n.reshape(-1)==i,:].reshape(-1,3)[:,j])
                    
                else:
                    a=np.quantile(m_true[true_n.reshape(-1)==i],0.001)
                    b=np.quantile(m_gen[gen_n.reshape(-1)==i],0.999)
  
                    h=hist.Hist(hist.axis.Regular(15,a,b,label=labels[j]))
                    h2=hist.Hist(hist.axis.Regular(15,a,b,label=labels[j]))
                    bins = h.axes[0].edges
                    h.fill(m_gen[gen_n.reshape(-1)==i])
                    h2.fill(m_true[true_n.reshape(-1)==i])
                    

                h.plot1d(    ax=ax[k//2,k%2],label="Flow Simulated")  # line or bar)
                h2.plot1d(    ax=ax[k//2,k%2],label="MC Generated")  # line or bar)
                ax[k//2,k%2].set_title("{} Distribution for jets with {} particles".format(labels[j],i))

                ax[k//2,k%2].set_xlabel(labels[j])
                ax[k//2,k%2].set_ylabel("Counts",fontweight="bold")
                ax[k//2,k%2].set_xlim(a,b)
                ax[k//2,k%2].legend()
                #plt.tight_layout(pad=2)

            if save:
                self.summary.add_figure("jet{}_{}".format(i//3+1,names[i%3]),fig,global_step=self.step)
                self.summary.close()
            else:
                plt.savefig("{}{}{}jet{}_{}".format(self.typ,self.c,self.p,i//3+1,names[j]))
                plt.show()

In [4]:
from sklearn.preprocessing import PowerTransformer
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)
    def inverse_transform(self,values):
        return (values *self.std)+self.mean
    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)
    def to(self,dev):
        self.std=self.std.to(dev)
        self.mean=self.mean.to(dev)
        return self
  

    
class JetNetDataloader(pl.LightningDataModule):
    '''This is more or less standard boilerplate coded that builds the data loader of the training
       one thing to note is the custom standard scaler that works on tensors
       Currently only jets with 30 particles are used but this maybe changes soon'''
    def __init__(self,batch_size,parton): 
        super().__init__()
        self.parton=parton
        self.batch_size=batch_size
    def setup(self,stage):
    # This just sets up the dataloader, nothing particularly important. it reads in a csv, calculates mass and reads out the number particles per jet
    # And adds it to the dataset as variable. The only important thing is that we add noise to zero padded jets
        data_dir=os.environ["HOME"]+"/JetNet_NF/train_{}_jets.csv".format(self.parton)
        data=pd.read_csv(data_dir,sep=" ",header=None)
        jets=[]
        
        for njets in range(1,31):
            masks=np.sum(data.values[:,np.arange(3,120,4)],axis=1)
            df=data.loc[masks==njets,:]
            df=df.drop(np.arange(3,120,4),axis=1)
            df["n"]=njets
            if len(df)>100:
                jets.append(df)
        #stacking together differnet samples with different number particles per jet
        self.n=torch.empty((0,1))
        
        self.data=torch.empty((0,90))
        self.masks=torch.empty((0,30))
        for i in range(len(jets)):
            
            x=torch.tensor(jets[i].drop(["n"],axis=1).values).float()
            n=torch.tensor(jets[i]["n"].values).int()
            
            m=torch.ones((len(jets[i]),30)).float()
            m[:,n[0]+1:]=0
                         
            self.data=torch.vstack((self.data,x))
            self.n=torch.vstack((self.n.reshape(-1,1),n.reshape(-1,1)))        
            self.masks=torch.vstack((self.masks,m))
            
#         if self.config["canonical"]:
#             self.data=preprocess(self.data)        
        # calculating mass per jet
        self.m=mass(self.data).reshape(-1,1) 
        # Adding noise to zero padded jets.
        for i in torch.unique(self.n):
            i=int(i)
            self.data[self.n.flatten()==i,3*i:90]=torch.normal(mean=torch.zeros_like(self.data[self.n.flatten()==i,3*i:90]),std=1).abs()*1e-7
        #standard scaling 
        shitscaling=False
        if shitscaling:
            self.scaler=StandardScaler()
            self.scaler2=PowerTransformer(method='box-cox')

    #         self.data=torch.hstack((self.data,self.m))        

            self.scaler.fit(self.data.reshape(-1,30,3)[:,:,:2].reshape(-1,60))
            self.scaler2.fit(self.data.reshape(-1,30,3)[:,:,2].reshape(-1,30))
            self.data=self.data.reshape(-1,30,3)
            self.data[:,:,:2]=self.scaler.transform(self.data[:,:,:2].reshape(-1,60)).reshape(-1,30,2)
            self.data[:,:,2]=torch.tensor(self.scaler2.transform(self.data[:,:,2].reshape(-1,30))).reshape(-1,30)
#         self.min_m=self.scaler.transform(torch.zeros((1,91)))[0,-1]
#         self.data=torch.hstack((self.data,self.n))
        else:
            self.scaler=StandardScaler()
            self.scaler.fit(self.data)
            self.data=self.scaler.transform(self.data)
        self.data=self.data.reshape(-1,90)
        
        #calculating mass dist in different bins, this is needed for the testcase where we need to generate the conditoon
#         if self.config["variable"]:
#             self.mdists={}
#             for i in torch.unique(self.n):
#                 self.mdists[int(i)]=F(self.data[self.n[:,0]==i,-2])    
        self.data,self.test_set,self.mask_data,self.mask_test=train_test_split(self.data.cpu().numpy(),self.masks.cpu().numpy(),test_size=0.3)
        
        self.n_train=self.data[:,-1]
        self.n_test=self.test_set[:,-1]
        self.test_set=torch.tensor(self.test_set).float()
        self.data=torch.tensor(self.data).float()
        self.data=torch.hstack((self.data,torch.tensor(self.mask_data)))
        self.test_set=torch.hstack((self.test_set,torch.tensor(self.mask_test)))
#         assert self.data.shape[1]==92
#         assert (torch.isnan(self.data)).sum()==0

    def train_dataloader(self):
        return DataLoader(self.data, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=len(self.test_set))
JetNetDataloader(1024,"t").setup("training")

In [5]:
from torch.autograd import Function
# class chamfer_3DFunction(Function):
#     @staticmethod
#     def forward(ctx, xyz1, xyz2):
#         batchsize, n, _ = xyz1.size()
#         _, m, _ = xyz2.size()
#         device = xyz1.device

#         dist1 = torch.zeros(batchsize, n)
#         dist2 = torch.zeros(batchsize, m)

#         idx1 = torch.zeros(batchsize, n).type(torch.IntTensor)
#         idx2 = torch.zeros(batchsize, m).type(torch.IntTensor)

#         dist1 = dist1.to(device)
#         dist2 = dist2.to(device)
#         idx1 = idx1.to(device)
#         idx2 = idx2.to(device)
#         torch.cuda.set_device(device)

#         chamfer_3D.forward(xyz1, xyz2, dist1, dist2, idx1, idx2)
#         ctx.save_for_backward(xyz1, xyz2, idx1, idx2)
#         return dist1, dist2, idx1, idx2

#     @staticmethod
#     def backward(ctx, graddist1, graddist2, gradidx1, gradidx2):
#         xyz1, xyz2, idx1, idx2 = ctx.saved_tensors
#         graddist1 = graddist1.contiguous()
#         graddist2 = graddist2.contiguous()
#         device = graddist1.device

#         gradxyz1 = torch.zeros(xyz1.size())
#         gradxyz2 = torch.zeros(xyz2.size())

#         gradxyz1 = gradxyz1.to(device)
#         gradxyz2 = gradxyz2.to(device)
#         chamfer_3D.backward(
#             xyz1, xyz2, gradxyz1, gradxyz2, graddist1, graddist2, idx1, idx2
#         )
#         return gradxyz1, gradxyz2
# class cd(nn.Module):
#     def __init__(self):
#         super(cd, self).__init__()

#     def forward(self, input1, input2):
#         input1 = input1.contiguous()
#         input2 = input2.contiguous()
#         return chamfer_3DFunction.apply(input1, input2)
def calc_dcd(x, gt, alpha=1000, n_lambda=1, return_raw=False, non_reg=False):
    x = x.float()
    gt = gt.float()
    batch_size, n_x, _ = x.shape
    batch_size, n_gt, _ = gt.shape
    assert x.shape[0] == gt.shape[0]

    if non_reg:
        frac_12 = max(1, n_x / n_gt)
        frac_21 = max(1, n_gt / n_x)
    else:
        frac_12 = n_x / n_gt
        frac_21 = n_gt / n_x

    cd_p, cd_t, dist1, dist2, idx1, idx2 = calc_cd(x, gt, return_raw=True)
    # dist1 (batch_size, n_gt): a gt point finds its nearest neighbour x' in x;
    # idx1  (batch_size, n_gt): the idx of x' \in [0, n_x-1]
    # dist2 and idx2: vice versa
    exp_dist1, exp_dist2 = torch.exp(-dist1 * alpha), torch.exp(-dist2 * alpha)

    count1 = torch.zeros_like(idx2)
    count1.scatter_add_(1, idx1.long(), torch.ones_like(idx1))
    weight1 = count1.gather(1, idx1.long()).float().detach() ** n_lambda
    weight1 = (weight1 + 1e-6) ** (-1) * frac_21
    loss1 = (1 - exp_dist1 * weight1).mean(dim=1)

    count2 = torch.zeros_like(idx1)
    count2.scatter_add_(1, idx2.long(), torch.ones_like(idx2))
    weight2 = count2.gather(1, idx2.long()).float().detach() ** n_lambda
    weight2 = (weight2 + 1e-6) ** (-1) * frac_12
    loss2 = (1 - exp_dist2 * weight2).mean(dim=1)

    loss = (loss1 + loss2) / 2

    res = [loss, cd_p, cd_t]
    if return_raw:
        res.extend([dist1, dist2, idx1, idx2])

    return res

def calc_cd(output, gt, calc_f1=False, return_raw=False, normalize=False, separate=False):
    # cham_loss = dist_chamfer_3D.chamfer_3DDist()
#     cham_loss = distChamfer(output,gt)
    dist1, dist2, idx1, idx2 = distChamfer(gt, output)
    cd_p = (torch.sqrt(dist1).mean(1) + torch.sqrt(dist2).mean(1)) / 2
    cd_t = (dist1.mean(1) + dist2.mean(1))

    if separate:
        res = [torch.cat([torch.sqrt(dist1).mean(1).unsqueeze(0), torch.sqrt(dist2).mean(1).unsqueeze(0)]),
               torch.cat([dist1.mean(1).unsqueeze(0),dist2.mean(1).unsqueeze(0)])]
    else:
        res = [cd_p, cd_t]
    if calc_f1:
        f1, _, _ = fscore(dist1, dist2)
        res.append(f1)
    if return_raw:
        res.extend([dist1, dist2, idx1, idx2])
    return res

def pairwise_dist(x, y):
    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = xx.diag().unsqueeze(0).expand_as(xx)
    ry = yy.diag().unsqueeze(0).expand_as(yy)
    P = rx.t() + ry - 2 * zz
    return P


def NN_loss(x, y, dim=0):
    dist = pairwise_dist(x, y)
    values, indices = dist.min(dim=dim)
    return values.mean()


def distChamfer(a, b):
    """
    :param a: Pointclouds Batch x nul_points x dim
    :param b:  Pointclouds Batch x nul_points x dim
    :return:
    -closest point on b of points from a
    -closest point on a of points from b
    -idx of closest point on b of points from a
    -idx of closest point on a of points from b
    Works for pointcloud of any dimension
    """
    x, y = a.double(), b.double()
    bs, num_points_x, points_dim = x.size()
    bs, num_points_y, points_dim = y.size()

    xx = torch.pow(x, 2).sum(2)
    yy = torch.pow(y, 2).sum(2)
    zz = torch.bmm(x, y.transpose(2, 1))
    rx = xx.unsqueeze(1).expand(bs, num_points_y, num_points_x)  # Diagonal elements xx
    ry = yy.unsqueeze(1).expand(bs, num_points_x, num_points_y)  # Diagonal elements yy
    P = rx.transpose(2, 1) + ry - 2 * zz
    return (
        torch.min(P, 2)[0].float(),
        torch.min(P, 1)[0].float(),
        torch.min(P, 2)[1].int(),
        torch.min(P, 1)[1].int(),
    )

In [6]:
def dcd(x, gt, alpha=1, lpnorm=2.0, n_lambda=1, non_reg=False):
    x = x.float()
    gt = gt.float()
    batch_size, n_x, _ = x.shape
    batch_size, n_gt, _ = gt.shape
    assert x.shape[0] == gt.shape[0]

    if non_reg:
        frac_12 = max(1, n_x / n_gt)
        frac_21 = max(1, n_gt / n_x)
    else:
        frac_12 = n_x / n_gt
        frac_21 = n_gt / n_x

    dist1, dist2, idx1, idx2 = distChamfer(x, gt, lpnorm=lpnorm)
    # dist1 (batch_size, n_gt): a gt point finds its nearest neighbour x' in x;
    # idx1  (batch_size, n_gt): the idx of x' \in [0, n_x-1]
    # dist2 and idx2: vice versa
    exp_dist1, exp_dist2 = torch.exp(-dist1 * alpha), torch.exp(-dist2 * alpha)

    count1 = torch.zeros_like(idx2)
    count1.scatter_add_(1, idx1.long(), torch.ones_like(idx1))
    weight1 = count1.gather(1, idx1.long()).float().detach() ** n_lambda
    weight1 = (weight1 + 1e-6) ** (-1) * frac_21
    loss1 = (1 - exp_dist1 * weight1).mean(dim=1)

    count2 = torch.zeros_like(idx1)
    count2.scatter_add_(1, idx2.long(), torch.ones_like(idx2))
    weight2 = count2.gather(1, idx2.long()).float().detach() ** n_lambda
    weight2 = (weight2 + 1e-6) ** (-1) * frac_12
    loss2 = (1 - exp_dist2 * weight2).mean(dim=1)

    return (loss1 + loss2) / 2


def cd(output, gt, lpnorm=2.0):
    dist1, dist2, _, _ = distChamfer2(gt, output, lpnorm=lpnorm)
    cd_p = (dist1.mean(1) + dist2.mean(1)) / 2
    return cd_p
#     cd_p = (torch.sqrt(dist1).mean(1) + torch.sqrt(dist2).mean(1)) / 2
#     cd_t = dist1.mean(1) + dist2.mean(1)

    # , calc_f1=False, separate=False
#     if separate:
#         res = [
#             torch.cat(
#                 [
#                     torch.sqrt(dist1).mean(1).unsqueeze(0),
#                     torch.sqrt(dist2).mean(1).unsqueeze(0),
#                 ]
#             ),
#             torch.cat([dist1.mean(1).unsqueeze(0), dist2.mean(1).unsqueeze(0)]),
#         ]
#     else:
#         res = [cd_p, cd_t]
#     if calc_f1:
#         f1, _, _ = fscore(dist1, dist2)
#         res.append(f1)
#     return res


def distChamfer2(a, b, lpnorm=2):
    """
    :param a: Pointclouds Batch x nul_points x dim
    :param b:  Pointclouds Batch x nul_points x dim
    :return:
    -closest point on b of points from a
    -closest point on a of points from b
    -idx of closest point on b of points from a
    -idx of closest point on a of points from b
    Works for pointcloud of any dimension
    """
    # original implementation equivalent to
    # P = torch.pow(torch.cdist(a, b, p=lpnorm), lpnorm)
    P = torch.cdist(a, b, p=lpnorm)
    return (
        torch.min(P, 2)[0],
        torch.min(P, 1)[0],
        torch.min(P, 2)[1].int(),
        torch.min(P, 1)[1].int(),
    )


def fscore(dist1, dist2, threshold=0.0001):
    """
    Calculates the F-score between two point clouds with the corresponding threshold value.
    :param dist1: Batch, N-Points
    :param dist2: Batch, N-Points
    :param th: float
    :return: fscore, precision, recall
    """
    # NB : In this depo, dist1 and dist2 are squared pointcloud
    # euclidean distances, so you should adapt the threshold accordingly.
    precision_1 = torch.mean((dist1 < threshold).float(), dim=1)
    precision_2 = torch.mean((dist2 < threshold).float(), dim=1)
    fscore = 2 * precision_1 * precision_2 / (precision_1 + precision_2)
    fscore[torch.isnan(fscore)] = 0
    return fscore, precision_1, precision_2

In [7]:
from torch.nn import MultiheadAttention,TransformerEncoder,TransformerEncoderLayer


In [8]:
  
class DecoderBlock(nn.Module):
    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn1 = MultiheadAttention(dim_feedforward, dim_feedforward, num_heads)
#         self.self_attn2 = MultiheadAttention(dim_feedforward, dim_feedforward, num_heads)
        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(dim_feedforward, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )
        
        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(dim_feedforward)
        self.norm2 = nn.LayerNorm(dim_feedforward)
        self.norm3 = nn.LayerNorm(dim_feedforward)
        self.dropout = nn.Dropout(dropout)

    def forward(self, z,e_out,src_mask=None,trg_mask=None):
        # Attention part
        attn_out = self.self_attn1(z,z,z, mask=trg_mask)
        z = z + self.dropout(attn_out)
        z = self.norm1(z)
#         z = z+self.self_attn2(z,e_out,e_out,src_mask)
#         z = self.norm2(z)
        # MLP part
        linear_out = self.linear_net(z)
#         z = z + self.dropout(linear_out)
        x = self.norm2(z)
        return x
    
    

    
class Decoder(nn.Module):
    def __init__(self, num_layers,input_dim, num_heads, dim_feedforward):
        super().__init__()
        self.layers = nn.ModuleList([DecoderBlock(input_dim, num_heads, dim_feedforward) for _ in range(num_layers)])

    def forward(self, z,e_outputs,src_mask=None,trg_mask=None):
        for l in self.layers:
            z = l(z,e_outputs,src_mask,trg_mask)
        return z




In [9]:
import copy
from mp import *
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

bce = torch.nn.BCELoss()
mse = torch.nn.MSELoss()

class TransformerPredictor(pl.LightningModule):

    def __init__(self, input_dim, model_dim, num_heads, num_layers, lr, warmup,max_iters, dropout=0.0, input_dropout=0.0):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()
        self.loss=lambda x,y: (x-y)**2
        self.metrics={"val_w1p":[],"val_w1m":[],"val_w1efp":[],"val_cov":[],"val_mmd":[],"val_fpnd":[],"val_logprob":[],"step":[]}

    def _create_model(self):
        self.encoder=TransformerEncoderLayer(d_model=3,nhead=3,dim_feedforward=128,activation="relu")
        self.encoder=TransformerEncoder(self.encoder,num_layers=6)
        self.disc=MPDiscriminator(num_particles=30,input_node_size=2)
#  
    def to(self,dev):
#         self.input_net=self.input_net.to(dev)

#         self.mu=self.mu.to(dev)
#         self.sigma=self.sigma.to(dev)
#         self.decoder=self.decoder.to(dev)
        self.encoder=self.encoder.to(dev)
        self.disc=self.disc.to(dev)
#         self.N.loc = self.N.loc.to(dev) # hack to get sampling on the GPU
#         self.N.scale = self.N.scale.to(dev)
        
    def calc_G_loss(self,fake_outputs):
        """Calculates generator loss for the different possible loss functions"""
        Y_real = torch.ones(fake_outputs.shape[0], 1, device=fake_outputs.device)
        loss="og"
        if loss == "og":
            G_loss = bce(fake_outputs, Y_real)
        elif loss == "ls":
            G_loss = mse(fake_outputs, Y_real)
        elif loss == "w" or loss == "hinge":
            G_loss = -fake_outputs.mean()

        return G_loss

    
    def calc_D_loss(self,
        real_outputs,
        fake_outputs,
        gp_lambda=0,
        
    ):
        """
        calculates discriminator loss for the different possible loss functions
        + optionally applying label smoothing, label flipping, or a gradient penalty
        returns individual loss contributions as well for evaluation and plotting
        """

        Y_real = torch.ones(len(real_outputs), 1).to(self.device)
        Y_fake = torch.zeros(len(fake_outputs), 1).to(self.device)

        loss="w" 
        if loss == "og":
            D_real_loss = bce(real_outputs, Y_real)
            D_fake_loss = bce(fake_outputs, Y_fake)
        elif loss == "ls":
            D_real_loss = mse(real_outputs, Y_real)
            D_fake_loss = mse(fake_outputs, Y_fake)
        elif loss == "w":
            D_real_loss = -real_outputs.mean()
            D_fake_loss = fake_outputs.mean()

        D_loss = D_real_loss + D_fake_loss

        if gp_lambda:
            gp = gradient_penalty(gp_lambda, D, data, gen_data, run_batch_size, device, model)
            gpitem = gp.item()
            D_loss += gp
        else:
            gpitem = None

        return (
            D_loss
    #         {
    #             "Dr": D_real_loss.item(),
    #             "Df": D_fake_loss.item(),
    #             "gp": gpitem,
    #             "D": D_real_loss.item() + D_fake_loss.item(),
    #         },
        )
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.disc.parameters(), lr=self.hparams.lr)
        optimizer_g = torch.optim.Adam(self.encoder.parameters(), lr=self.hparams.lr)
        # Apply lr scheduler per step
        lr_scheduler = CosineWarmupScheduler(optimizer,
                                             warmup=self.hparams.warmup,
                                             max_iters=self.hparams.max_iters)
        return optimizer,optimizer_g#],[{'scheduler': lr_scheduler, 'interval': 'step'}]



    def load_datamodule(self,data_module):
        '''needed for lightning training to work, it just sets the dataloader for training and validation'''
        self.data_module=data_module   

    
    
    def training_step(self, batch, batch_idx,optimizer_idx):
        self.to("cuda")
        d_opt, _opt = self.optimizers()
        batch,mask=batch[:,:90],batch[:,-30:]
        batch=batch[:,:90].reshape(-1,30,3)
        gen=self.encoder(batch)#,z,trg_mask=None

        y_real=self.disc(batch)
        y_gen=self.disc(gen)
        if optimizer_idx==0:

            l_d=self.calc_D_loss(y_real,y_gen)
            print(l_d)
            return l_d
#         l=l.mean()#+self.kl
        if optimizer_idx==1:
            l_g=self.calc_G_loss(y_gen)
            print(l_g)
            return l_g
#         self.log("loss",l,logger=True,on_step=True,on_epoch=True)
       # self.log("kl",l,logger=True,on_step=True,on_epoch=True)
        return OrderedDict({"loss":l})
    
    
    def validation_step(self, batch, batch_idx):
        '''This calculates some important metrics on the hold out set (checking for overtraining)'''
        self.data_module.scaler.to("cpu")  
        self.to("cpu")
        batch,mask=batch[:,:90],batch[:,-30:]
        batch=batch.to("cpu")
        mask=mask.to("cpu")
        mask,trg_mask=self.create_masks(mask,mask)
        mask=mask.unsqueeze(1).unsqueeze(1)
        trg_mask=trg_mask.unsqueeze(1).unsqueeze(1)
        z=torch.randn((len(batch),30,3))
        self.n_dim=90
        
        
        with torch.no_grad():
            test=self.forward(batch[:,:90].reshape(-1,30,3)).reshape(-1,90)#,mask
            loss=self.loss(test.reshape(-1,30,3),batch.reshape(-1,30,3))
            loss=loss[0].mean()+loss[1].mean()
        self.log("val_loss",loss,logger=True)
        assert (test==test).all()
        # Reverse Standard Scaling (this has nothing to do with flows, it is a standard preprocessing step)
        shitscaling=False
        if shitscaling:
            test=test.reshape(-1,30,3)

            test[:,:,-1]=torch.tensor(self.data_module.scaler2.inverse_transform(test.reshape(-1,30,3)[:,:,2].reshape(-1,30)).reshape(-1,30))
            test[:,:,:2]=self.data_module.scaler.inverse_transform(test.reshape(-1,30,3)[:,:,:2].reshape(-1,60)).reshape(-1,30,2)
        # gen=self.data_module.scaler.inverse_transform(gen)
            test=test.reshape(-1,90)
            true=batch.reshape(-1,30,3)
            true[:,:,-1]=torch.tensor(self.data_module.scaler2.inverse_transform(true.reshape(-1,30,3)[:,:,2].reshape(-1,30)).reshape(-1,30))
            true[:,:,:2]=self.data_module.scaler.inverse_transform(true.reshape(-1,30,3)[:,:,:2].reshape(-1,60)).reshape(-1,30,2)
            true=true.reshape(-1,90)
        else:
            test=self.data_module.scaler.inverse_transform(test)
            true=self.data_module.scaler.inverse_transform(batch)
            
            
        #         true=self.data_module.scaler2.inverse_transform(batch.numpy())[:,:90]
        # We overwrite in cases where n is smaller 30 the particles after n with 0
        # if self.config["context_features"]>1:
        #     for i in torch.unique(batch[:,-1]):
        #         i=int(i)
        #         gen[c[:,-1]==i,3*i:]=0
        #         test[c_test[:,-1]==i,3*i:-1]=0
        #This is just a nice check to see whether we overtrain
        m_t=mass(true[:,:self.n_dim].to(self.device)).cpu()
        # m_gen=mass(gen[:,:self.n_dim],self.config["canonical"]).cpu()
        m_test=mass(test[:,:self.n_dim]).cpu()
        # gen=torch.column_stack((gen[:,:90],m_gen))
        test=torch.column_stack((test[:,:90],m_test))       
        # Again checking for overtraining
        mse=FF.mse_loss(m_t,m_test).detach()
       
        # For one metric the pt needs to always be bigger or equal 0, so we overwrite the cases where it isnt (its not physical possible to ahve pt smaller 0)
        for i in range(30):
            i=2+3*i
            # gen[gen[:,i]<0,i]=0
            test[test[:,i]<0,i]=0
            true[true[:,i]<0,i]=0
          #Some metrics we track
        cov,mmd=cov_mmd(test[:,:self.n_dim].reshape(-1,self.n_dim//3,3),true[:,:self.n_dim].reshape(-1,self.n_dim//3,3),use_tqdm=False)

        fpndv=fpnd(test[:,:self.n_dim].reshape(-1,self.n_dim//3,3).numpy(),use_tqdm=False,jet_type="t")

        self.metrics["val_fpnd"].append(fpndv)
        self.metrics["val_mmd"].append(mmd)
        self.metrics["val_cov"].append(cov)
        self.metrics["val_w1p"].append(w1p(test[:,:self.n_dim].reshape(-1,self.n_dim//3,3),true[:,:self.n_dim].reshape(-1,self.n_dim//3,3)))
        self.metrics["val_w1m"].append(w1m(test[:,:self.n_dim].reshape(-1,self.n_dim//3,3),true[:,:self.n_dim].reshape(-1,self.n_dim//3,3)))
        try:
            self.metrics["val_w1efp"].append(w1efp(test[:,:self.n_dim].reshape(-1,self.n_dim//3,3),true[:,:self.n_dim].reshape(-1,self.n_dim//3,3)))
        except:
            self.metrics["val_w1efp"]=None
        self.log("val_w1m",self.metrics["val_w1m"][-1][0],on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_w1p",self.metrics["val_w1p"][-1][0],on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_w1efp",self.metrics["val_w1efp"][-1][0],on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_cov",cov,prog_bar=True,logger=True,on_step=False, on_epoch=True)
        self.log("val_fpnd",fpndv,prog_bar=True,logger=True,on_step=False, on_epoch=True)
        self.log("val_mmd",mmd,prog_bar=True,logger=True,on_step=False, on_epoch=True)
        self.log("val_mse",mse,prog_bar=True,logger=True,on_step=False, on_epoch=True)
        # This part here adds the plots to tensorboard
        self.plot=plotting(model=self,gen=test[:,:self.n_dim],true=true[:,:self.n_dim],step=self.global_step,logger=self.logger.experiment)
        
        try:
            self.plot.plot_mass(m_test.cpu().numpy(),m_t.cpu().numpy(),save=True,bins=15,quantile=True,plot_vline=False)
#             self.plot.plot_marginals(save=True)
#             self.plot.plot_2d(save=True)
#             self.plot.losses(save=True)
#             self.plot.var_part(true=true[:,:self.n_dim],gen=test[:,:self.n_dim],true_n=n_true,gen_n=n_test,m_true=m_t,m_gen=m_test ,save=True)
        except Exception as e:
            traceback.print_exc()
            
# #         self.flow.to("cuda")
#         self.plot.plot_correlations()

    def test_step(self, batch, batch_idx):
        raise NotImplementedError

In [10]:
from torch.nn import functional as F

data_module = JetNetDataloader(120,"t") #this loads the data
model = TransformerPredictor(input_dim=3,max_iters=10000, 
                             model_dim=3, num_heads=1, num_layers=1, lr=0.01, warmup=50,
                             dropout=0.0, input_dropout=0.0) # the sets up the model,  config are hparams we want to optimize
# Callbacks to use during the training, we  checkpoint our models

# if True:#load_ckpt:
#     model = model.load_from_checkpoint("/beegfs/desy/user/kaechben/t/2022_06_22-10_50-16/epoch=1749-val_logprob=0.94-val_w1m=0.0153.ckpt")
model.load_datamodule(data_module)#adds datamodule to model


#log every n steps could be important as it decides how often it should log to tensorboard
# Also check val every n epochs, as validation checking takes some time
logger = TensorBoardLogger("./")
trainer = pl.Trainer(gpus=1,logger=logger,   # auto_scale_batch_size="binsearch",
                      max_steps=50000, num_sanity_val_steps=0,
                      check_val_every_n_epoch=100,track_grad_norm=2,
                     fast_dev_run=False,max_epochs=-1)
# This calls the fit function which trains the model
trainer.fit(model, train_dataloaders=data_module )  


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type               | Params
-----------------------------------------------
0 | encoder | TransformerEncoder | 5.8 K 
1 | disc    | MPDiscriminator    | 355 K 
-----------------------------------------------
360 K     Trainable params
0         Non-trainable params
360 K     Total params
1.444     Total estimated model params size (MB)
/beegfs/desy/user/kaechben/.conda/envs/jetnet/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/beegfs/desy/user/kaechben/.conda/en

Training: 0it [00:00, ?it/s]

tensor(-0.0648, device='cuda:0', grad_fn=<AddBackward0>)


/opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [102,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [103,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [104,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [107,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [32,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/opt/conda/conda-bld/pytorch_1646755903507/wor

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [104]:
import gc
del model
torch.cuda.empty_cache()
gc.collect()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
import torch
from torch import nn


class Encoder(nn.Module):
    def __init__(self,c) -> None:
        super().__init__()

#         self.point_encoding = nn.Sequential(
#             nn.Conv1d(3, 6, kernel_size=1),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv1d(6, 8, kernel_size=1),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv1d(8, 16, kernel_size=1),
#             nn.LeakyReLU(0.2, inplace=True),
#         )  
#         self.latent_encoding = nn.Sequential(
#             nn.Linear(16, 16),
#             # nn.BatchNorm1d(),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(16, 16),
#             # nn.BatchNorm1d(),
            
#             nn.LeakyReLU(0.2, inplace=True),
#         )
        
        self.point_encoding = nn.Sequential(
            nn.Linear(90, 60),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(60, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 16),
            nn.LeakyReLU(0.2, inplace=True),
        )  
        # self.mu_mapping = nn.Linear(32, 32)
        #self.logvar_mapping = nn.Linear(32, 32)
        self.latent_rep = nn.Linear(16, c)

    def forward(self, points: torch.Tensor) -> torch.Tensor: # Tuple[torch.Tensor, torch.Tensor]:
        points = self.point_encoding(points.reshape(-1,90))
        # use channel as latent space
        # summing is perm invariant
        #latent = points.sum(axis=2)
        #latent = self.latent_encoding(latent)
        #mu = self.mu_mapping(latent)
        #logvar = self.logvar_mapping(latent)
        #return mu, logvar

        return self.latent_rep(points)

    
